In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ec163e77affc46fa99f98234ef367f9b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fAoov85MUzPeW32wmnh7O3aCEBEs-W7G4SqaeZS9Go-3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_ec163e77affc46fa99f98234ef367f9b.get_object(Bucket='edx1-donotdelete-pr-mimwnjmslwrxhv',Key='311_Service_Requests_from_2010_to_Present_min.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Unnamed: 0,Unique Key,Created Date,Closed Date,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,Address Type,City,Status,Resolution Description,Borough,Latitude,Longitude
0,0,45531130,02/02/2020 06:09:17 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019.0,426 WEST 52 STREET,WEST 52 STREET,ADDRESS,NEW YORK,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,1,45529784,02/02/2020 02:15:24 PM,NaN,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204.0,1751 67 STREET,67 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,2,45527528,02/02/2020 02:27:41 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372.0,87-15 37 AVENUE,37 AVENUE,ADDRESS,Jackson Heights,Open,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,3,45530329,02/02/2020 12:13:18 PM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458.0,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,ADDRESS,BRONX,Open,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,4,45528814,02/02/2020 01:59:44 PM,NaN,APPLIANCE,RESIDENTIAL BUILDING,11209.0,223 78 STREET,78 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533


In [2]:
client_cred = ibm_boto3.client(service_name='s3',
ibm_api_key_id='fAoov85MUzPeW32wmnh7O3aCEBEs-W7G4SqaeZS9Go-3',
ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
config=Config(signature_version='oauth'),
endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

In [3]:
df_data_1.to_pickle('./df_raw.pkl')

In [4]:
client_cred.upload_file('./df_raw.pkl','edx1-donotdelete-pr-mimwnjmslwrxhv','df_raw_cos.pkl')

In [5]:
client_cred.download_file(Bucket='edx1-donotdelete-pr-mimwnjmslwrxhv',Key='df_raw_cos.pkl',Filename='./df_raw_local.pkl')

In [6]:
df_raw_local = pd.read_pickle('./df_raw_local.pkl')
#df_raw_local.head()
df_raw_local.shape

(6019843, 16)

In [7]:
df_raw_local.columns

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Location Type', 'Incident Zip', 'Incident Address',
       'Street Name', 'Address Type', 'City', 'Status',
       'Resolution Description', 'Borough', 'Latitude', 'Longitude'],
      dtype='object')

In [8]:
df_raw_local.dtypes

Unnamed: 0                  int64
Unique Key                  int64
Created Date               object
Closed Date                object
Complaint Type             object
Location Type              object
Incident Zip              float64
Incident Address           object
Street Name                object
Address Type               object
City                       object
Status                     object
Resolution Description     object
Borough                    object
Latitude                  float64
Longitude                 float64
dtype: object

In [9]:
#Choose 'Complaint Type'  Columns to counts the number of each complaint types
temp_complaint_type = df_raw_local['Complaint Type'].value_counts()
#sorted(temp_complaint_type)
#temp_complaint_type.dtype
complaint_type = temp_complaint_type.sort_values(ascending=True)
complaint_type.count()

30

In [10]:
import matplotlib.pyplot as plt
#Visulizing the complaints type and number of complainted in horizontal plot
complaint_type.plot(kind='barh', figsize=(12, 12), color='red',fontsize=15)
plt.xlabel('Number of Complaints',fontsize=15)
plt.title('Complaints type', fontsize=15)

#Annotate value labels to each country
for index, value in enumerate(complaint_type): 
    label = format(int(value), ',') # format int with commas
    
    # place text at the end of bar (subtracting 0.1 from y to make it fit within the bar)
    plt.annotate(label, xy=(value, index-0.1 ), color='black', fontsize=15)

plt.show()

<Figure size 1200x1200 with 1 Axes>

In [11]:
import numpy as np
#After reading the New york city data file, one can see that HEAT/HOT WATER complaint column has been merged with HEATING after 2014. Lets merge them together
#df_raw_local['Complaint Type'] = np.where(df_raw_local['Complaint Type']=='HEATING','HEAT/HOT WATER',df_raw_local['Complaint Type'])
#df_raw_local['Complaint Type'].value_counts()

In [12]:
#from datetime import datetime
#df_raw_local['Created Date'] = pd.to_datetime(df_raw_local['Created Date'])
#df_raw_local.info()

In [13]:
#one can observe HEATING complaint type being merged with HEAT/HOT WATER complaint type and 'created_date' format is datetime

#df_raw_local['created_year'] = df_raw_local['Created Date'].map(lambda x: x.strftime('%Y')).astype(int)

#df_raw_local['created_month'] = df_raw_local['Created Date'].map(lambda x: x.strftime('%m')).astype(int)

#fig, ax = plt.subplots(figsize=(18,9))

#df_raw_local.groupby(['created_year', 'complaint_type']).count()['unique_key'].unstack().plot(ax=ax)

#fig, ax = plt.subplots(figsize=(18,9))

#df_raw_local.groupby(['Created Date', 'Complaint Type']).count()['Unique Key'].unstack().plot(ax=ax)

In [14]:
#fig, ax = plt.subplots(figsize=(18,9))

#df[df['created_month'].isin([11, 12,1])].groupby(['created_year', 'complaint_type']).count()['unique_key'].unstack().plot.bar(ax=ax)

#<font color=blue> Looking at the above bar plot, it is clear the Heating complaints is heighest. Here we have specifically focussed on the winter season.
#<font color=red> Concluding remarks: Solution of problem 1: Based on the above plot it is clear that maximum number of complaints are coming fromHEAT/HOT water category. So HPD should address the HEAT/HOT WATER complaint first. The problem remails all time high. It is clear with the time dependent plots.

In [15]:
#Question 1 - What is the total number of complaints that exist in the dataset?
df_raw_local['Complaint Type'].count()

6019843

In [16]:
#Question 1 - What is the total number of complaints that exist in the dataset?
df_raw_local['Complaint Type'].notnull().sum().sum()

6019843

In [17]:
#Question 2 - How many differnt Complaint Types can you find in the dataset, including duplicates entries of the same type?
df_raw_local['Complaint Type'].value_counts().count()

30

In [18]:
#Question 3 - How many Elevator complaints can you find in the dataset?
complaint_type = df_raw_local['Complaint Type'].value_counts()
complaint_type
#df_raw_local[df_raw_local["Complaint Type"] == "SEDGWICK AVENUE"]["Complaint Type"].value_counts()

HEAT/HOT WATER            1261574
HEATING                    887850
PLUMBING                   711130
GENERAL CONSTRUCTION       500863
UNSANITARY CONDITION       451643
PAINT - PLASTER            361257
PAINT/PLASTER              346438
ELECTRIC                   307310
NONCONST                   260890
DOOR/WINDOW                205278
WATER LEAK                 193631
GENERAL                    151308
FLOORING/STAIRS            137402
APPLIANCE                  112831
HPD Literature Request      52824
SAFETY                      51529
OUTSIDE BUILDING             7142
ELEVATOR                     6725
Unsanitary Condition         5499
CONSTRUCTION                 5078
General                      1163
Safety                        424
STRUCTURAL                     16
Plumbing                       11
AGENCY                          9
VACANT APARTMENT                6
Outside Building                6
Appliance                       4
Mold                            1
Electric      

In [19]:
#Question 3 - How many Elevator complaints can you find in the dataset?
df_raw_local[df_raw_local["Complaint Type"] == "ELEVATOR"]["Complaint Type"].value_counts()

ELEVATOR    6725
Name: Complaint Type, dtype: int64

In [20]:
#Question 4 - How many Electric complaints can you find in the dataset?
df_raw_local[df_raw_local["Complaint Type"] == "ELECTRIC"]["Complaint Type"].value_counts()
#note = calculate Electric as well

ELECTRIC    307310
Name: Complaint Type, dtype: int64

In [21]:
import numpy as np
df_raw_local['Complaint Type'] = np.where(df_raw_local['Complaint Type']=='HEATING','HEAT/HOT WATER',df_raw_local['Complaint Type'])
df_raw_local['Complaint Type'].value_counts()

HEAT/HOT WATER            2149424
PLUMBING                   711130
GENERAL CONSTRUCTION       500863
UNSANITARY CONDITION       451643
PAINT - PLASTER            361257
PAINT/PLASTER              346438
ELECTRIC                   307310
NONCONST                   260890
DOOR/WINDOW                205278
WATER LEAK                 193631
GENERAL                    151308
FLOORING/STAIRS            137402
APPLIANCE                  112831
HPD Literature Request      52824
SAFETY                      51529
OUTSIDE BUILDING             7142
ELEVATOR                     6725
Unsanitary Condition         5499
CONSTRUCTION                 5078
General                      1163
Safety                        424
STRUCTURAL                     16
Plumbing                       11
AGENCY                          9
VACANT APARTMENT                6
Outside Building                6
Appliance                       4
Mold                            1
Electric                        1
Name: Complain

In [22]:
#df = df_raw_local.loc[:,['Borough', 'City', 'Incident Zip']]
df_borough = df_raw_local.loc[:,['Borough']]
df_borough

,Borough
0,MANHATTAN
1,BROOKLYN
2,QUEENS
3,BRONX
4,BROOKLYN
5,BRONX
6,BROOKLYN
7,QUEENS
8,BROOKLYN
9,BROOKLYN


In [23]:
temp_borough = df_borough['Borough'].value_counts()
temp_borough

BROOKLYN         1739886
BRONX            1617956
MANHATTAN        1055225
Unspecified       873221
QUEENS            645971
STATEN ISLAND      87584
Name: Borough, dtype: int64

In [24]:
import numpy as np
#df_raw_local = df_raw_local.loc[df_raw_local['Complaint Type']=='HEAT/HOT WATER'].dropna()
#df_raw_local = df_raw_local.loc[df_raw_local['Complaint Type']=='HEATING'].dropna()
#df_raw_local['Complaint Type'] = np.where(df_raw_local['Complaint Type']=='HEATING','HEAT/HOT WATER',df_raw_local['Complaint Type'])
#df_raw_local['Complaint Type'].value_counts()
#df_raw_local = df_raw_local.dropna()
#df_raw_local.head()

In [25]:
#df_raw_local = df_raw_local.dropna()
#df_raw_local.head()